In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

# import vital data

In [2]:
df_vital = pd.read_csv("Archive/sepsis2_task1_training/sepsis2_task1_vital_training.csv")
df_vital.head()

FileNotFoundError: [Errno 2] File b'Archive/sepsis2_task1_training/sepsis2_task1_vital_training.csv' does not exist: b'Archive/sepsis2_task1_training/sepsis2_task1_vital_training.csv'

In [94]:
df_vital[df_vital['adm_id'] == 'A100001']

,adm_id,event_time,A/G Ratio,ALT/SGPT,AST/SGOT,Albumin Quant,"Albumin, Serum","Alk Phos, Serum","Amylase, Serum",Anion Gap,...,Troponin I,Troponin T S/Pl,UA Red Blood Cell,UA Urobilinogen,UA WBC,"Vitamin B12, Serum",WBC,Weight,"pH, Arterial","pH, Urine"
0,A100001,1.5,NaN,22.0,30.0,NaN,3.9,102.0,NaN,12.0,...,NaN,NaN,NaN,NaN,NaN,NaN,4.7,NaN,NaN,NaN
1,A100001,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,A100001,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,A100001,4.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,A100001,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# reinvent an inefficient wheel for vital data processing (negelect it)

In [241]:
# column_names = list(df_vital.columns.values)
# column_names.remove('event_time')
# adm_id_list = sorted(set(df_vital.adm_id.values))#get unduplicated adm id'scolumn_names = list(df_vital.columns.values)
# df_vital_new = pd.DataFrame({'adm_id':adm_id_list},list(range(len(adm_id_list))),columns = column_names) # new dataframe with unduplicated adm id's
# for id_index,id_item in enumerate(adm_id_list): #iterate over adm id's
#     if id_index<-1 or id_index>1:
#         continue
#     if id_index%10 == 0:
#         print("{:.9f}% Processed".format((id_index+1)/len(adm_id_list))) #approximate progress bar
#     item_data = df_vital[df_vital['adm_id'] == id_item].sort_values(by = ['event_time'],ascending = True) #sort data by event_time in case not sorted
#     for row_index in reversed(range(len(item_data))): # iterate each adm_id's data from the latest to the earlist
#         for col in df_vital_new:#check NaN for each coloumn and do substitution
#             if pd.isnull(df_vital_new.loc[id_index][col]):
#                 if pd.isnull(item_data.iloc[row_index][col]):
#                     pass
#                 else:
#                     df_vital_new.at[id_index,col] = item_data.iloc[row_index][col]
# df_vital_new.to_csv('Archive/sepsis2_task1_training/df_vital_new.csv',index = False)

0.000009408% Processed


# process vital data assuming Malkov property and stationary assumption

In [108]:
import time
t1 = time.time()
df_vital_new = df_vital.groupby('adm_id').last().reset_index()
t2 = time.time()
print(t2-t1)
df_vital_new = df_vital_new.drop(columns = ['event_time'])
df_vital_new.to_csv('Archive/sepsis2_task1_training/df_vital_new.csv',index = False) #save the data

20.661787748336792


In [96]:
df_vital_new.head(20)

,adm_id,A/G Ratio,ALT/SGPT,AST/SGOT,Albumin Quant,"Albumin, Serum","Alk Phos, Serum","Amylase, Serum",Anion Gap,BMI,...,Troponin I,Troponin T S/Pl,UA Red Blood Cell,UA Urobilinogen,UA WBC,"Vitamin B12, Serum",WBC,Weight,"pH, Arterial","pH, Urine"
0,A100001,NaN,22.0,30.00,NaN,3.90,102.00,NaN,12.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,4.70,NaN,NaN,NaN
1,A100002,NaN,54.5,52.50,NaN,2.00,138.00,239.0,7.0,25.00,...,NaN,NaN,NaN,NaN,NaN,NaN,19.42,88.586518,NaN,NaN
2,A100004,NaN,22.0,26.00,NaN,3.00,63.00,NaN,10.0,31.62,...,NaN,NaN,NaN,NaN,NaN,722.0,9.90,94.330000,NaN,NaN
3,A100005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,25.00,...,NaN,NaN,NaN,NaN,NaN,NaN,11.70,68.000000,NaN,NaN
4,A100006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,4.60,NaN,NaN,NaN
5,A100019,NaN,45.0,73.00,NaN,2.70,NaN,NaN,NaN,23.60,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75.600000,NaN,NaN
6,A100023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.40,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,63.000000,NaN,NaN
7,A100030,1.0,55.0,237.00,NaN,2.10,20.00,NaN,9.0,NaN,...,NaN,NaN,2.44,NaN,NaN,470.0,5.10,NaN,7.52,5.5
8,A100032,NaN,72.0,51.00,NaN,3.00,95.00,NaN,3.0,30.49,...,NaN,NaN,NaN,NaN,NaN,NaN,8.30,82.003145,NaN,NaN
9,A100033,NaN,NaN,NaN,NaN,2.30,NaN,NaN,6.0,36.10,...,NaN,NaN,NaN,NaN,NaN,742.0,1.80,104.540000,NaN,NaN


# load all data for training

In [4]:
###
#load data
###
df_label = pd.read_csv("Archive/sepsis2_task1_training/sepsis2_task1_label_training.csv")
df_vital_new = pd.read_csv("Archive/sepsis2_task1_training/df_vital_new.csv")
df_master = pd.read_csv("Archive/sepsis2_task1_training/sepsis2_task1_master_training.csv")

# create the complete version of dataframe (vital, master, label)

In [5]:
###
#create dummy variables for master dataframe
###
gender_dum = pd.get_dummies(df_master.gender,prefix = 'gender')
race_dum = pd.get_dummies(df_master.race,prefix = 'race')
adtype_dum = pd.get_dummies(df_master.admission_type,prefix = 'adtype')
adsource_dum = pd.get_dummies(df_master.admission_source,prefix = 'adsource')
care_dum = pd.get_dummies(df_master.care_setting,prefix = 'care_setting')
age_dum = pd.get_dummies(df_master.age_grp,prefix = 'age_grp')
df_master = pd.concat([df_master,gender_dum,race_dum,adtype_dum,adsource_dum,care_dum,age_dum],axis = 1)
df_master = df_master.drop(columns = ['gender','race','admission_type','admission_source','care_setting','age_grp'])
df_master.head()

,adm_id,gender_Female,gender_Male,race_African American,race_Asian,race_Caucasian,race_Others/unknown,adtype_Elective,adtype_Emergency,adtype_Others/unknown,...,care_setting_Step-Down Unit - Cardiac,care_setting_Surgery,age_grp_20~30,age_grp_30~40,age_grp_40~50,age_grp_50~60,age_grp_60~70,age_grp_70~80,age_grp_<20,age_grp_>=80
0,A100001,0,1,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
1,A100002,0,1,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,A100004,0,1,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
3,A100005,1,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
4,A100006,1,0,0,0,1,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0


In [6]:
# merge all dataframes together
df_label_vital = pd.merge(df_master,df_vital_new,on = 'adm_id')
df_final = pd.merge(df_label_vital,df_label,on = 'adm_id')
df_final = df_final.drop(columns = ['adm_id'])
df_final.to_csv('Archive/sepsis2_task1_training/Markov_stationary.csv',index=False) #save the data
df_final.head(10)

,gender_Female,gender_Male,race_African American,race_Asian,race_Caucasian,race_Others/unknown,adtype_Elective,adtype_Emergency,adtype_Others/unknown,adtype_Trauma Center,...,Troponin T S/Pl,UA Red Blood Cell,UA Urobilinogen,UA WBC,"Vitamin B12, Serum",WBC,Weight,"pH, Arterial","pH, Urine",sepsis2
0,0,1,1,0,0,0,0,1,0,0,...,NaN,NaN,NaN,NaN,NaN,4.70,NaN,NaN,NaN,0
1,0,1,0,0,1,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,19.42,88.586518,NaN,NaN,0
2,0,1,0,0,1,0,0,1,0,0,...,NaN,NaN,NaN,NaN,722.0,9.90,94.330000,NaN,NaN,0
3,1,0,0,0,1,0,0,1,0,0,...,NaN,NaN,NaN,NaN,NaN,11.70,68.000000,NaN,NaN,0
4,1,0,0,0,1,0,1,0,0,0,...,NaN,NaN,NaN,NaN,NaN,4.60,NaN,NaN,NaN,0
5,0,1,0,0,1,0,1,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,75.600000,NaN,NaN,1
6,1,0,0,0,1,0,0,1,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,63.000000,NaN,NaN,0
7,0,1,0,0,1,0,0,1,0,0,...,NaN,2.44,NaN,NaN,470.0,5.10,NaN,7.52,5.5,0
8,1,0,1,0,0,0,0,1,0,0,...,NaN,NaN,NaN,NaN,NaN,8.30,82.003145,NaN,NaN,1
9,1,0,0,0,1,0,0,1,0,0,...,NaN,NaN,NaN,NaN,742.0,1.80,104.540000,NaN,NaN,0


# Random forest training

In [7]:
# df_final = pd.read_csv('Archive/sepsis2_task1_training/Markov_stationary.csv')
y = df_final.sepsis2
x_data = df_final.drop(columns=["sepsis2"])
x = (x_data - np.min(x_data))/(np.max(x_data)-np.min(x_data)) #normalization
x = x.fillna(x.mean())
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.2, random_state = 0) #split train and test data

In [9]:
import time
t1 = time.time()
rf = RandomForestClassifier(n_estimators = 1000, random_state = 1, verbose = 2)
rf.fit(x_train, y_train)
acc = rf.score(x_test,y_test)*100
t2 = time.time()
print("Random Forest Algorithm Accuracy Score : {:.2f}%".format(acc))
print(t2-t1)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


building tree 1 of 1000


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.5s remaining:    0.0s


building tree 2 of 1000
building tree 3 of 1000
building tree 4 of 1000
building tree 5 of 1000
building tree 6 of 1000
building tree 7 of 1000
building tree 8 of 1000
building tree 9 of 1000
building tree 10 of 1000
building tree 11 of 1000
building tree 12 of 1000
building tree 13 of 1000
building tree 14 of 1000
building tree 15 of 1000
building tree 16 of 1000
building tree 17 of 1000
building tree 18 of 1000
building tree 19 of 1000
building tree 20 of 1000
building tree 21 of 1000
building tree 22 of 1000
building tree 23 of 1000
building tree 24 of 1000
building tree 25 of 1000
building tree 26 of 1000
building tree 27 of 1000
building tree 28 of 1000
building tree 29 of 1000
building tree 30 of 1000
building tree 31 of 1000
building tree 32 of 1000
building tree 33 of 1000
building tree 34 of 1000
building tree 35 of 1000
building tree 36 of 1000
building tree 37 of 1000
building tree 38 of 1000
building tree 39 of 1000
building tree 40 of 1000
building tree 41 of 1000
building

building tree 322 of 1000
building tree 323 of 1000
building tree 324 of 1000
building tree 325 of 1000
building tree 326 of 1000
building tree 327 of 1000
building tree 328 of 1000
building tree 329 of 1000
building tree 330 of 1000
building tree 331 of 1000
building tree 332 of 1000
building tree 333 of 1000
building tree 334 of 1000
building tree 335 of 1000
building tree 336 of 1000
building tree 337 of 1000
building tree 338 of 1000
building tree 339 of 1000
building tree 340 of 1000
building tree 341 of 1000
building tree 342 of 1000
building tree 343 of 1000
building tree 344 of 1000
building tree 345 of 1000
building tree 346 of 1000
building tree 347 of 1000
building tree 348 of 1000
building tree 349 of 1000
building tree 350 of 1000
building tree 351 of 1000
building tree 352 of 1000
building tree 353 of 1000
building tree 354 of 1000
building tree 355 of 1000
building tree 356 of 1000
building tree 357 of 1000
building tree 358 of 1000
building tree 359 of 1000
building tre

building tree 638 of 1000
building tree 639 of 1000
building tree 640 of 1000
building tree 641 of 1000
building tree 642 of 1000
building tree 643 of 1000
building tree 644 of 1000
building tree 645 of 1000
building tree 646 of 1000
building tree 647 of 1000
building tree 648 of 1000
building tree 649 of 1000
building tree 650 of 1000
building tree 651 of 1000
building tree 652 of 1000
building tree 653 of 1000
building tree 654 of 1000
building tree 655 of 1000
building tree 656 of 1000
building tree 657 of 1000
building tree 658 of 1000
building tree 659 of 1000
building tree 660 of 1000
building tree 661 of 1000
building tree 662 of 1000
building tree 663 of 1000
building tree 664 of 1000
building tree 665 of 1000
building tree 666 of 1000
building tree 667 of 1000
building tree 668 of 1000
building tree 669 of 1000
building tree 670 of 1000
building tree 671 of 1000
building tree 672 of 1000
building tree 673 of 1000
building tree 674 of 1000
building tree 675 of 1000
building tre

building tree 954 of 1000
building tree 955 of 1000
building tree 956 of 1000
building tree 957 of 1000
building tree 958 of 1000
building tree 959 of 1000
building tree 960 of 1000
building tree 961 of 1000
building tree 962 of 1000
building tree 963 of 1000
building tree 964 of 1000
building tree 965 of 1000
building tree 966 of 1000
building tree 967 of 1000
building tree 968 of 1000
building tree 969 of 1000
building tree 970 of 1000
building tree 971 of 1000
building tree 972 of 1000
building tree 973 of 1000
building tree 974 of 1000
building tree 975 of 1000
building tree 976 of 1000
building tree 977 of 1000
building tree 978 of 1000
building tree 979 of 1000
building tree 980 of 1000
building tree 981 of 1000
building tree 982 of 1000
building tree 983 of 1000
building tree 984 of 1000
building tree 985 of 1000
building tree 986 of 1000
building tree 987 of 1000
building tree 988 of 1000
building tree 989 of 1000
building tree 990 of 1000
building tree 991 of 1000
building tre

[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:  7.8min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


Random Forest Algorithm Accuracy Score : 78.34%
482.0166370868683


[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:   14.0s finished


# save model

In [157]:
import _pickle

with open('random_forest.pickle', 'wb') as f:
    _pickle.dump(rf, f)


# in your prediction file                                                                                                                                                                                                           

# with open('model/random_forest.cpicke', 'rb') as f:
#     rf = cPickle.load(f)


# acc = rf.score(x_test,y_test)*100
# t2 = time.time()
# print("Random Forest Algorithm Accuracy Score : {:.2f}%".format(acc))

OSError: [Errno 28] No space left on device

In [11]:
feature_importances = pd.DataFrame(rf.feature_importances_,index = x_train.columns,columns=['importance']).sort_values('importance',ascending=False)
feature_importances

,importance
HR Monitored,0.045982
Resp Rt,0.045892
HR,0.032399
Temp,0.021883
WBC,0.021395
SBP,0.018787
DBP,0.018572
Glasgow Coma Score,0.018168
Weight,0.016800
Height,0.016701


In [163]:
feature_importances.to_csv('Archive/sepsis2_task1_training/feature_importances.csv')

In [ ]:
feature_importances = pd.read_csv('Archive/sepsis2_task1_training/feature_importances.csv')

4988

In [194]:
features_to_drop = feature_importances.index.values[15:]

In [195]:
df_final_dropped = df_final.drop(columns = features_to_drop)

In [196]:
# df_final = pd.read_csv('Archive/sepsis2_task1_training/Markov_stationary.csv')
y = df_final_dropped.sepsis2
x_data = df_final_dropped.drop(columns=["sepsis2"])
x = (x_data - np.min(x_data))/(np.max(x_data)-np.min(x_data)) #normalization
x = x.fillna(x.mean())
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.2, random_state = 0) #split train and test data

import time
t1 = time.time()
rf_dropped = RandomForestClassifier(n_estimators = 1000, random_state = 1, verbose = 2)
rf_dropped.fit(x_train, y_train)
acc = rf_dropped.score(x_test,y_test)*100
t2 = time.time()
print("Random Forest Algorithm Accuracy Score : {:.2f}%".format(acc))
print(t2-t1)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s


building tree 1 of 1000
building tree 2 of 1000
building tree 3 of 1000
building tree 4 of 1000
building tree 5 of 1000
building tree 6 of 1000
building tree 7 of 1000
building tree 8 of 1000
building tree 9 of 1000
building tree 10 of 1000
building tree 11 of 1000
building tree 12 of 1000
building tree 13 of 1000
building tree 14 of 1000
building tree 15 of 1000
building tree 16 of 1000
building tree 17 of 1000
building tree 18 of 1000
building tree 19 of 1000
building tree 20 of 1000
building tree 21 of 1000
building tree 22 of 1000
building tree 23 of 1000
building tree 24 of 1000
building tree 25 of 1000
building tree 26 of 1000
building tree 27 of 1000
building tree 28 of 1000
building tree 29 of 1000
building tree 30 of 1000
building tree 31 of 1000
building tree 32 of 1000
building tree 33 of 1000
building tree 34 of 1000
building tree 35 of 1000
building tree 36 of 1000
building tree 37 of 1000
building tree 38 of 1000
building tree 39 of 1000
building tree 40 of 1000
building 

building tree 321 of 1000
building tree 322 of 1000
building tree 323 of 1000
building tree 324 of 1000
building tree 325 of 1000
building tree 326 of 1000
building tree 327 of 1000
building tree 328 of 1000
building tree 329 of 1000
building tree 330 of 1000
building tree 331 of 1000
building tree 332 of 1000
building tree 333 of 1000
building tree 334 of 1000
building tree 335 of 1000
building tree 336 of 1000
building tree 337 of 1000
building tree 338 of 1000
building tree 339 of 1000
building tree 340 of 1000
building tree 341 of 1000
building tree 342 of 1000
building tree 343 of 1000
building tree 344 of 1000
building tree 345 of 1000
building tree 346 of 1000
building tree 347 of 1000
building tree 348 of 1000
building tree 349 of 1000
building tree 350 of 1000
building tree 351 of 1000
building tree 352 of 1000
building tree 353 of 1000
building tree 354 of 1000
building tree 355 of 1000
building tree 356 of 1000
building tree 357 of 1000
building tree 358 of 1000
building tre

building tree 637 of 1000
building tree 638 of 1000
building tree 639 of 1000
building tree 640 of 1000
building tree 641 of 1000
building tree 642 of 1000
building tree 643 of 1000
building tree 644 of 1000
building tree 645 of 1000
building tree 646 of 1000
building tree 647 of 1000
building tree 648 of 1000
building tree 649 of 1000
building tree 650 of 1000
building tree 651 of 1000
building tree 652 of 1000
building tree 653 of 1000
building tree 654 of 1000
building tree 655 of 1000
building tree 656 of 1000
building tree 657 of 1000
building tree 658 of 1000
building tree 659 of 1000
building tree 660 of 1000
building tree 661 of 1000
building tree 662 of 1000
building tree 663 of 1000
building tree 664 of 1000
building tree 665 of 1000
building tree 666 of 1000
building tree 667 of 1000
building tree 668 of 1000
building tree 669 of 1000
building tree 670 of 1000
building tree 671 of 1000
building tree 672 of 1000
building tree 673 of 1000
building tree 674 of 1000
building tre

building tree 953 of 1000
building tree 954 of 1000
building tree 955 of 1000
building tree 956 of 1000
building tree 957 of 1000
building tree 958 of 1000
building tree 959 of 1000
building tree 960 of 1000
building tree 961 of 1000
building tree 962 of 1000
building tree 963 of 1000
building tree 964 of 1000
building tree 965 of 1000
building tree 966 of 1000
building tree 967 of 1000
building tree 968 of 1000
building tree 969 of 1000
building tree 970 of 1000
building tree 971 of 1000
building tree 972 of 1000
building tree 973 of 1000
building tree 974 of 1000
building tree 975 of 1000
building tree 976 of 1000
building tree 977 of 1000
building tree 978 of 1000
building tree 979 of 1000
building tree 980 of 1000
building tree 981 of 1000
building tree 982 of 1000
building tree 983 of 1000
building tree 984 of 1000
building tree 985 of 1000
building tree 986 of 1000
building tree 987 of 1000
building tree 988 of 1000
building tree 989 of 1000
building tree 990 of 1000
building tre

[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:  2.4min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


Random Forest Algorithm Accuracy Score : 77.28%
150.7930998802185


[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    5.3s finished


# Regression

In [36]:
# df_final = pd.read_csv('Archive/sepsis2_task1_training/Markov_stationary.csv')
y = df_final.sepsis2
x_data = df_final.drop(columns=["sepsis2"])
x = (x_data - np.min(x_data))/(np.max(x_data)-np.min(x_data)) #normalization
x = x.fillna(0)
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.2, random_state = 0) #split train and test data

In [44]:
lr = LogisticRegression(penalty = 'elasticnet',verbose = 2,solver = 'saga', l1_ratio = 0.5)
lr.fit(x_train,y_train)
lr.score(x_test,y_test)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


convergence after 85 epochs took 18 seconds


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   17.9s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   17.9s finished


0.7537513523684086

In [74]:
coefs = (lr.coef_ > 1) | (lr.coef_ < -1)
coef_names = []
for i in range(len(coefs[0])):
    if coefs[0][i] == True:
        coef_names.append(df_final.columns[i])

In [75]:
df_regress = df_final[coef_names]
# df_final = pd.read_csv('Archive/sepsis2_task1_training/Markov_stationary.csv')

,care_setting_Physical Rehabilitation,AST/SGOT,Anion Gap,Baso Abs Cnt Bld,"Bilirubin Direct, Serum",Bilirubin Total Bld mCnc,Body Surface Area,"Calcium, Serum","Creatinine, Serum Quant",Diff Bands,...,PT,PT Time PPP,PTT,Pain Score,Resp Rt,"Triglyceride, Serum",Troponin I,Troponin T S/Pl,WBC,"pH, Urine"
0,0,30.000000,12.000000,0.00,0.10,0.400000,NaN,8.800000,1.110000,NaN,...,NaN,NaN,NaN,NaN,18.0,85.0,NaN,NaN,4.700000,NaN
1,0,52.500000,7.000000,0.01,0.13,0.450000,NaN,7.550000,0.700000,9.0,...,11.7,NaN,NaN,NaN,16.0,81.0,NaN,NaN,19.420000,NaN
2,0,26.000000,10.000000,0.00,0.20,0.500000,2.0800,8.300000,1.750000,NaN,...,24.0,NaN,NaN,0.0,20.0,122.0,NaN,NaN,9.900000,NaN
3,0,NaN,10.000000,NaN,NaN,NaN,NaN,8.300000,0.670000,NaN,...,NaN,NaN,NaN,0.0,16.0,NaN,NaN,NaN,11.700000,NaN
4,0,NaN,8.000000,NaN,NaN,NaN,NaN,8.000000,0.580000,NaN,...,15.1,NaN,NaN,NaN,16.0,NaN,NaN,NaN,4.600000,NaN
5,0,73.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,6.0,17.0,NaN,NaN,NaN,NaN,NaN
6,0,NaN,NaN,NaN,NaN,NaN,1.6600,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,31.0,NaN,NaN,NaN,NaN,NaN
7,0,237.000000,9.000000,0.00,2.50,3.700000,NaN,7.400000,0.540000,NaN,...,18.6,NaN,26.7,NaN,20.0,NaN,NaN,NaN,5.100000,5.5
8,0,51.000000,3.000000,NaN,NaN,1.000000,1.9300,7.300000,0.780000,NaN,...,13.7,NaN,NaN,NaN,18.0,NaN,NaN,NaN,8.300000,NaN
9,0,NaN,6.000000,0.00,NaN,NaN,NaN,7.800000,1.100000,2.0,...,NaN,NaN,NaN,6.0,20.0,NaN,NaN,NaN,1.800000,NaN


In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
imp = IterativeImputer(max_iter=10, random_state=0)
imp.fit(df_final)

In [43]:
for k in [0.2, 0.4, 0.6, 0.8]:
    lr = LogisticRegression(penalty = 'elasticnet',verbose = 2,solver = 'saga', l1_ratio = k)
    lr.fit(x_train,y_train)
    print(lr.score(x_test,y_test))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


convergence after 67 epochs took 13 seconds
0.7540806246766075


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   13.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   13.4s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


convergence after 77 epochs took 16 seconds
0.7537983912695799


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   15.7s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   15.7s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


convergence after 99 epochs took 20 seconds
0.7538454301707512


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   19.6s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   19.6s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


max_iter reached after 20 seconds
0.7544098969848064


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   20.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   20.3s finished
